## US county-level sociodemographic and health resource data (2018-2019)
Sociodemographic and health resource data have been collected by county in the United States and we want to find out if there is any relationship between health resources and sociodemographic data.

To do this, you need to set a target variable (health-related) to conduct the analysis.

In [118]:
# Step 0. Import libraries, custom modules and logging
# Basics ---------------------------------------------------------------
import logging
import joblib
# Data -----------------------------------------------------------------
import pandas as pd
import numpy as np
# Graphics -------------------------------------------------------------
import matplotlib.pyplot as plt
import seaborn as sns
# Machine learning -----------------------------------------------------
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import (
    OneHotEncoder,
    StandardScaler
)
from sklearn.compose import ColumnTransformer
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.metrics import (mean_squared_error, 
                             mean_absolute_error,
                             r2_score)
from sklearn.model_selection import GridSearchCV
# Logging initialization -----------------------------------------------
logger = logging.getLogger()
logger.setLevel(logging.INFO)

In [119]:
# 1 Create data frame
# 1.1 Read from source and get basic info
df_raw = pd.read_csv("https://raw.githubusercontent.com/4GeeksAcademy/regularized-linear-regression-project-tutorial/main/demographic_health_data.csv", sep = ",")
df_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3140 entries, 0 to 3139
Columns: 108 entries, fips to Urban_rural_code
dtypes: float64(61), int64(45), object(2)
memory usage: 2.6+ MB


In [120]:
df_raw.sample(10, random_state=2024)

,fips,TOT_POP,0-9,0-9 y/o % of total pop,19-Oct,10-19 y/o % of total pop,20-29,20-29 y/o % of total pop,30-39,30-39 y/o % of total pop,...,COPD_number,diabetes_prevalence,diabetes_Lower 95% CI,diabetes_Upper 95% CI,diabetes_number,CKD_prevalence,CKD_Lower 95% CI,CKD_Upper 95% CI,CKD_number,Urban_rural_code
2190,40123,38247,5275,13.791931,5247,13.718723,5736,14.997255,4855,12.693806,...,2498,11.9,11.1,12.7,3445,3.2,3.0,3.4,925,5
284,8083,26158,3138,11.996330,3147,12.030736,2672,10.214848,2990,11.430537,...,1678,11.4,10.4,12.3,2321,3.6,3.4,3.9,740,6
2514,47177,40878,5029,12.302461,5476,13.395959,4773,11.676207,4915,12.023582,...,4040,15.3,14.1,16.5,4797,3.8,3.5,4.1,1181,5
2476,47101,12086,1346,11.136853,1508,12.477246,1351,11.178223,1315,10.880357,...,1224,15.5,14.3,16.7,1468,3.8,3.5,4.2,364,6
1479,28161,12392,1488,12.007747,1540,12.427373,1486,11.991607,1314,10.603615,...,992,17.1,16.1,18.1,1645,3.9,3.7,4.2,375,6
572,16047,15196,2250,14.806528,2325,15.300079,1800,11.845222,1728,11.371414,...,882,12.7,11.8,13.6,1401,3.4,3.2,3.7,378,6
2956,53011,481857,61427,12.747973,64952,13.479518,60445,12.544178,64304,13.345038,...,21744,9.6,8.8,10.4,35011,2.7,2.5,2.9,9872,2
3090,55089,89147,9764,10.952696,11737,13.165895,9341,10.478199,9934,11.143392,...,3663,8.4,7.5,9.3,5861,2.7,2.5,2.9,1889,2
1457,28117,25315,3115,12.304958,3269,12.913293,3550,14.023306,2972,11.740075,...,2322,15.2,14.2,16.2,2999,3.8,3.6,4.2,757,6
1551,29141,20358,2548,12.515964,2510,12.329305,2006,9.853620,2013,9.888005,...,2118,15.5,14.1,16.8,2431,4.2,3.8,4.5,652,6


In [121]:
 #Step 2. Prepare the dataset for analysis
df_interim = (
    df_raw
    .copy()
    .dropna().reset_index(drop = True)
    .drop_duplicates().reset_index(drop = True)
    .set_axis(
        df_raw.columns.str.replace(' ', '_') # reemplazar espacio por _
        .str.replace(r'\W','',regex=True) # reemplazar caracteres no alfabéticos por nada
        .str.lower() # el resultado lo pasamos a minúsculas
        .str.slice(0,40), axis=1, # slice sirve para cortar el texto a un máximo de 40 caracteres
    )
    .rename(columns={'heart_disease_number':'target'})
    #.iloc[:,1:]
    .astype({"county_name": "category", "state_name": "category"})
)
df_interim.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3140 entries, 0 to 3139
Columns: 108 entries, fips to urban_rural_code
dtypes: category(2), float64(61), int64(45)
memory usage: 2.6 MB


In [122]:
df_interim.sample(10)

,fips,tot_pop,09,09_yo__of_total_pop,19oct,1019_yo__of_total_pop,2029,2029_yo__of_total_pop,3039,3039_yo__of_total_pop,...,copd_number,diabetes_prevalence,diabetes_lower_95_ci,diabetes_upper_95_ci,diabetes_number,ckd_prevalence,ckd_lower_95_ci,ckd_upper_95_ci,ckd_number,urban_rural_code
896,20021,20015,2374,11.861104,2676,13.369973,2104,10.512116,2287,11.426430,...,1481,14.0,12.9,15.1,2157,3.6,3.3,3.8,549,6
1004,21027,20388,2476,12.144399,2629,12.894840,2162,10.604277,2225,10.913282,...,2068,15.5,14.3,16.7,2433,3.6,3.4,4.0,572,6
2613,48185,28360,3365,11.865303,3618,12.757405,3506,12.362482,3518,12.404795,...,2106,15.5,14.6,16.4,3409,3.8,3.6,4.0,833,6
199,6031,151366,23256,15.364084,21654,14.305723,26099,17.242313,23433,15.481020,...,6984,11.5,11.0,12.2,12751,3.1,2.9,3.3,3412,4
2688,48335,8145,992,12.179251,967,11.872314,1444,17.728668,1170,14.364641,...,466,12.7,12.0,13.4,812,3.1,2.9,3.3,198,6
1582,29205,6050,777,12.842975,807,13.338843,595,9.834711,722,11.933884,...,491,13.2,12.1,14.2,606,3.5,3.3,3.8,161,6
1003,21025,12726,1429,11.228980,1403,11.024674,1581,12.423385,1482,11.645450,...,1726,17.6,16.1,18.9,1784,4.3,3.9,4.7,433,6
2585,48129,3319,348,10.485086,511,15.396204,394,11.871045,324,9.761976,...,242,13.6,12.5,14.6,362,3.6,3.4,3.9,97,6
304,8123,314305,45797,14.570879,45965,14.624330,43011,13.684478,47099,14.985126,...,12435,8.9,8.3,9.6,20680,2.6,2.4,2.7,5933,3
1991,38007,919,114,12.404788,83,9.031556,86,9.357998,140,15.233950,...,50,11.0,9.8,12.1,80,3.1,2.8,3.4,23,6


In [123]:
data_types = df_interim.dtypes
numeric_columns = [c for c in list(data_types[data_types != "category"].index) if c != "target"]

scaler = StandardScaler()
norm_features = scaler.fit_transform(df_interim[numeric_columns])

# Create a new DataFrame with the scaled numerical variables
df_interim_scal = pd.DataFrame(norm_features, index = df_interim.index, columns = numeric_columns)
df_interim_scal["target"] = df_interim["target"]
df_interim_scal.head()

,fips,tot_pop,09,09_yo__of_total_pop,19oct,1019_yo__of_total_pop,2029,2029_yo__of_total_pop,3039,3039_yo__of_total_pop,...,diabetes_prevalence,diabetes_lower_95_ci,diabetes_upper_95_ci,diabetes_number,ckd_prevalence,ckd_lower_95_ci,ckd_upper_95_ci,ckd_number,urban_rural_code,target
0,-1.940874,-0.145679,-0.142421,0.158006,-0.135556,0.573496,-0.153144,0.027610,-0.139384,0.588469,...,-0.063696,-0.071720,-0.089834,-0.129902,-0.609615,-0.582796,-0.669652,-0.147523,-1.082865,3345
1,-1.940742,0.341296,0.287476,-0.242861,0.320383,-0.193107,0.183774,-0.469965,0.230620,-0.110300,...,-0.394103,-0.414900,-0.337677,0.376251,-0.433549,-0.393279,-0.343373,0.389791,-0.420704,13414
2,-1.940610,-0.237785,-0.239429,-0.419441,-0.246181,-0.439718,-0.225971,0.272104,-0.218759,0.656538,...,2.432709,2.483064,2.317776,-0.183415,1.855312,1.880929,1.777443,-0.204321,0.903618,2159
3,-1.940478,-0.245223,-0.246032,-0.426966,-0.254791,-0.609076,-0.230792,0.396168,-0.220555,1.264959,...,0.376846,0.423984,0.299632,-0.229096,-0.257483,-0.203761,-0.180233,-0.242100,-1.745026,1533
4,-1.940346,-0.138966,-0.135053,0.186249,-0.137140,0.216679,-0.155888,-0.200808,-0.143570,0.088582,...,0.156575,0.195197,0.158008,-0.111247,-0.081417,-0.014244,-0.017093,-0.124105,-1.745026,4101


In [124]:
# 2.2 Create the final dataset with the target in front, show result
df = (
    df_interim_scal
    .copy()
    .reindex(
        columns=(
            ['target'] +
            [c for c in df_interim.columns.to_list() if c not in ['target']]
        )
    )
)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3140 entries, 0 to 3139
Columns: 108 entries, target to urban_rural_code
dtypes: float64(107), int64(1)
memory usage: 2.6 MB


In [125]:
df.sample(10)

,target,fips,tot_pop,09,09_yo__of_total_pop,19oct,1019_yo__of_total_pop,2029,2029_yo__of_total_pop,3039,...,copd_number,diabetes_prevalence,diabetes_lower_95_ci,diabetes_upper_95_ci,diabetes_number,ckd_prevalence,ckd_lower_95_ci,ckd_upper_95_ci,ckd_number,urban_rural_code
1143,862,-0.550341,-0.278920,-0.268639,0.785011,-0.284074,-0.290530,-0.255848,1.192377,-0.252730,...,-0.303425,2.102302,2.139884,2.034527,-0.259718,2.031378,2.070446,1.777443,-0.269011,0.903618
2514,3235,1.107420,-0.189822,-0.184478,0.203136,-0.186670,0.386470,-0.195610,-0.194437,-0.183767,...,-0.113706,0.817388,0.767164,0.866128,-0.152255,0.622848,0.554308,0.635465,-0.166283,0.241457
3060,2551,1.625899,-0.182836,-0.196200,-0.643229,-0.196060,-0.517889,-0.185119,-0.010856,-0.171048,...,-0.214928,-1.128339,-1.177523,-1.116609,-0.197197,-0.785681,-0.772313,-0.669652,-0.182456,-1.082865
416,542,-1.144473,-0.292451,-0.282658,0.961937,-0.294576,0.580732,-0.274650,0.184603,-0.267735,...,-0.328046,1.808607,1.911097,1.751279,-0.283483,2.031378,2.070446,1.940583,-0.289841,0.903618
1640,1139,-0.020639,-0.276873,-0.278424,-1.208603,-0.287835,-1.416907,-0.274750,-1.633934,-0.263815,...,-0.303615,0.890812,0.805295,0.972346,-0.262575,1.855312,1.691412,1.940583,-0.262671,0.903618
1291,62896,-0.282321,3.462989,3.020688,-0.391643,3.296943,-0.310512,2.966031,0.173145,2.996755,...,3.828641,-1.128339,-1.063129,-1.152015,3.047457,-1.313880,-1.151348,-1.322211,3.216736,-1.745026
166,4202,-1.669289,-0.120497,-0.113403,0.296151,-0.097498,0.965566,-0.087278,1.140335,-0.127074,...,-0.078651,-0.210544,-0.186114,-0.196052,-0.104827,-0.609615,-0.582796,-0.506512,-0.118154,0.241457
3023,2248,1.562261,-0.237902,-0.238329,-0.324134,-0.250818,-0.864189,-0.239124,-0.560953,-0.229620,...,-0.195778,0.927523,0.843426,0.972346,-0.209802,0.446782,0.364791,0.635465,-0.223728,0.903618
699,867,-0.818096,-0.276615,-0.271726,-0.135049,-0.281543,-0.228484,-0.264906,-0.341850,-0.257976,...,-0.302470,0.376846,0.347722,0.370444,-0.269332,0.094650,0.175273,0.146046,-0.276515,0.903618
2234,2027,0.703410,-0.232070,-0.238760,-0.738105,-0.250794,-1.307305,-0.238640,-0.776906,-0.223720,...,-0.238404,0.009727,-0.033589,0.087196,-0.217768,0.446782,0.364791,0.472326,-0.215447,0.903618


In [126]:
# Step 3. Perform EDA
# 3.1 Split the dataset
df_train, df_test = train_test_split(df,
                                     random_state=2024,
                                     test_size=0.2)
df_train = df_train.reset_index(drop=True).sort_values(by='target')
df_train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2512 entries, 2166 to 230
Columns: 108 entries, target to urban_rural_code
dtypes: float64(107), int64(1)
memory usage: 2.1 MB


In [129]:
display(df_train.describe(include="number").T)

,count,mean,std,min,25%,50%,75%,max
target,2512.0,5737.457006,16204.592599,7.000000,800.500000,1832.000000,4443.250000,434790.000000
fips,2512.0,-0.005942,0.993040,-1.940874,-0.806775,-0.083419,0.966677,1.692838
tot_pop,2512.0,0.008918,1.044298,-0.312120,-0.279472,-0.233937,-0.102344,29.986269
09,2512.0,0.007908,1.041848,-0.304787,-0.274040,-0.230602,-0.104844,28.600342
09_yo__of_total_pop,2512.0,0.001053,0.995148,-5.589683,-0.599294,-0.037893,0.504059,6.398903
...,...,...,...,...,...,...,...,...
ckd_prevalence,2512.0,-0.008313,1.003230,-2.898476,-0.609615,-0.081417,0.622848,4.848438
ckd_lower_95_ci,2512.0,-0.008661,1.004255,-2.857003,-0.582796,-0.014244,0.554308,4.913206
ckd_upper_95_ci,2512.0,-0.007936,1.002743,-2.953608,-0.669652,-0.017093,0.635465,4.713958
ckd_number,2512.0,0.008131,1.041878,-0.318692,-0.278455,-0.224892,-0.085809,30.443001


In [130]:
# 3.3 Create univariate analysis- numerical features
df_train.hist(figsize=(150,150))
plt.show()

In [132]:
# Step 4. Create a machine learning model
# 4.1 Split features and reponse
X_train = df_train.reset_index(drop=True).drop('target', axis=1)
y_train = df_train['target'].reset_index(drop=True)
X_test = df_test.reset_index(drop=True).drop('target', axis=1)
y_test = df_test['target'].reset_index(drop=True)

In [138]:
nan_per_column = X_train.isna().sum().sum()
print(nan_per_column)

5024


In [133]:
# 4.3 Create the model with Linear Regression
linear_regression = LinearRegression()

In [134]:
linear_regression.fit(X_train, y_train)

ValueError: Input X contains NaN.
LinearRegression does not accept missing values encoded as NaN natively. For supervised learning, you might want to consider sklearn.ensemble.HistGradientBoostingClassifier and Regressor which accept missing values encoded as NaNs natively. Alternatively, it is possible to preprocess the data, for instance by using an imputer transformer in a pipeline or drop samples with missing values. See https://scikit-learn.org/stable/modules/impute.html You can find a list of all estimators that handle NaN values at the following page: https://scikit-learn.org/stable/modules/impute.html#estimators-that-handle-nan-values